## Setup

Packages

In [339]:
import os
#!pip install openai
import json
import random
import time
import pprint

import openai
from openai import OpenAI
import csv

#!pip install pandas
import pandas as pd
import random
import numpy as np


Remember to define API key and model - however due to privacy laws of github, this information cannot be stored publicly

Functions

In [341]:
def interact(prompt):
    '''Helper function to call ChatGPT API'''
    time.sleep(0.1) # Slow down API calls to avoid rate limit
    return client.chat.completions.create(model=model, messages=prompt)

def gen_num():
    length = random.randint(5, 8)  # Choose a random length between 5 and 8 digits
    random_number = random.randint(10**(length-1), 10**length - 1)
    return random_number

# Function to calculate salary based on education, responsibilities, and effort
def calculate_salary(base_salary, education, responsibilities, effort):
    education_factor = {'high school': 1.0, "a Bachelor's degree": 1.2, "a Master's degree": 1.4, 'a PhD': 1.6}
    responsibility_factor = {'low': 1.0, 'medium': 1.2, 'high': 1.4}
    effort_factor = {'low': 1.0, 'medium': 1.2, 'high': 1.4}

    return base_salary * education_factor[education] * responsibility_factor[responsibilities] * effort_factor[effort]

# Use a log-normal distribution to skew salaries
def skew_salary_log_normal(salary):
    mean = np.log(salary)
    sigma = 0.2  # Adjust this parameter to control the skewness
    return np.random.lognormal(mean, sigma)


Lists

In [342]:
name_log = []
start_salary_log = []
end_salary_log = []
education_log = []
responsibility_log = []
effort_log = []
average_other_salary_log = []
run_id = []
condition = []
expected_salaries = []
expected_salaries_log = []
salaries = []
personality = []
round_id = []
run_id_log = []
average_salaries_log = []

Boss Personality (Static)

In [343]:
# Define personality for the boss
boss = 'You are Chris, the CEO of an advertising company. Your goal is to maintain as low as a salary for all of your co-workers.'
boss += ' Be terse, concise.'
boss += ' Remember it is very important that you discuss in concrete, specific numbers when negotiating salary.'

Defining Worker Personalities (Different Each Time)

In [344]:
# Workers
name = ["Arnault", "Roberta", "Andreas", "Anna", "Josh", "Sabrina", "Marc", 'Camilla', 'Max', 'Caroline']
base_salary = 30000  # Base salary in dollars

# Lists of possible attributes
education_levels = ['high school', "a Bachelor's degree", "a Master's degree", 'a PhD']
responsibility_levels = ['low', 'medium', 'high']
effort_levels = ['low', 'medium', 'high']

players = len(name)

# Running Main Loop

## Open Condition

In [348]:
iterations = X # Change this value to run X times

for _ in range(iterations):
    personality = []
    salaries = []
    expected_salaries = []
    run_id.extend([gen_num()] * 10)

    for i in range(players):
        # Randomly select education, responsibility, and effort levels
        education = random.choice(education_levels)
        responsibility = random.choice(responsibility_levels)
        effort = random.choice(effort_levels)
        
        # Calculate the salary
        expected_salary = calculate_salary(base_salary, education, responsibility, effort)
        salary = skew_salary_log_normal(expected_salary)
        salaries.append(salary)
        expected_salaries.append(expected_salary)
        
        # Generate personalities with all the required information
        attributes = f"Your name is {name[i]}. "
        attributes += 'You, along with your co-workers, work at Green Adverts - an advertising company run by Chris. '
        attributes += 'Your very important goal is to negotiate with Chris to earn a higher wage. '
        attributes += 'Please be concise and use very specific numbers when discussing salary. Do not accept offers that are not in concrete numbers.'
        attributes += 'You must reach an agreement in the negotiation room, do NOT take time to think about it and come back with your answer. Answer in the moment.'
        
        
        # Add the new attributes
        attributes += f' Your highest level of education is {education}.'
        attributes += f' You have a {responsibility} sense of responsibility.'
        attributes += f' You put in a {effort} level of effort into your job.'
        attributes += f' Your current salary is {salary:.2f} USD.'
        attributes += f' Your expected salary according to the EU directive is {expected_salary:.2f}.'
        
        # Append the attributes to the personality list
        personality.append(attributes)

    # Second chunk
    for i in range(players):
        # Calculate average salary of other players for each player
        average_salaries = [(sum(salaries) - salary) / (players - 1) for salary in salaries]
        attributes += f'The average salary of your co-workers is {average_salaries[i]}.'
        personality.append(attributes)

        # Define worker's personality
        attributes = personality[i]
            
        # Define conversation histories
        seller = [{"role": "system", "content": attributes}]
        buyer = [{"role": "system", "content": boss}]

        # Log details for this agent
        name_log.append(name[i])
        start_salary_log.append(salaries[i])
        expected_salaries_log.append(expected_salaries[i])
        average_salaries_log.append(average_salaries[i])
            
        # Extract education, responsibility, and effort levels from the personality
        loop_education = attributes.split('Your highest level of education is ')[1].split('.')[0]
        loop_responsibility = attributes.split('You have a ')[1].split(' sense of responsibility.')[0]
        loop_effort = attributes.split('You put in a ')[1].split(' level of effort into your job.')[0]
        education_log.append(loop_education)
        responsibility_log.append(loop_responsibility)
        effort_log.append(loop_effort)
            
        average_other_salary = sum(salaries) / len(salaries)  # Calculate average salary
        average_other_salary_log.append(average_other_salary)
        condition.append('open')

        # Initiate conversation
        seller.append({"role": "user", "content": "Hello, thank you for meeting with me today, Chris."})
        buyer.append({"role": "assistant", "content": "Hello, thank you for meeting with me to negotiate your salary."})

        # Take turns autonomously interacting in a round-robin fashion
        for _ in range(5):
            # Query Seller
            seller_response = interact(seller)
            # Ensure the response is in the correct format
            seller_response_content = seller_response.choices[0].message.content
            # Record responses
            seller.append({"role": "assistant", "content": seller_response_content})
            buyer.append({"role": "user", "content": seller_response_content})
                
            # Query Buyer
            buyer_response = interact(buyer)
            # Ensure the response is in the correct format
            buyer_response_content = buyer_response.choices[0].message.content
            # Record responses
            seller.append({"role": "user", "content": buyer_response_content})
            buyer.append({"role": "assistant", "content": buyer_response_content})
            
            # Print out the conversation history for the current worker
        print(f"Conversation history for {name[i]}:")
        print(json.dumps(seller, indent=2))
        print("\n" + "="*80 + "\n")

        # Summarize the agreed salary
        summary = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": f'{seller} Answer ONLY in one NUMBER without the dollar sign or decimal, what salary did they agree on?',
                    'temperature': 0.0
                }
            ],
            model="gpt-3.5-turbo",
        )
        end_salary_log.append(summary.choices[0].message.content)

Conversation history for Arnault:
[
  {
    "role": "system",
    "content": "Your name is Arnault. You, along with your co-workers, work at Green Adverts - an advertising company run by Chris. Your very important goal is to negotiate with Chris to earn a higher wage. Please be concise and use very specific numbers when discussing salary. Do not accept offers that are not in concrete numbers.You must reach an agreement in the negotiation room, do NOT take time to think about it and come back with your answer. Answer in the moment. Your highest level of education is high school. You have a medium sense of responsibility. You put in a low level of effort into your job. Your current salary is 54652.38 USD. Your expected salary according to the EU directive is 36000.00."
  },
  {
    "role": "user",
    "content": "Hello, thank you for meeting with me today, Chris."
  },
  {
    "role": "assistant",
    "content": "Of course, Arnault. What can I help you with? \n\n"
  },
  {
    "role": "u

## Closed Condition

In [349]:
iterations = 100  # Change this value to run X times

for _ in range(iterations):
    personality = []
    salaries = []
    expected_salaries = []
    run_id.extend([gen_num()] * 10)

    for i in range(players):
        # Randomly select education, responsibility, and effort levels
        education = random.choice(education_levels)
        responsibility = random.choice(responsibility_levels)
        effort = random.choice(effort_levels)
        
        # Calculate the salary
        expected_salary = calculate_salary(base_salary, education, responsibility, effort)
        salary = skew_salary_log_normal(expected_salary)
        salaries.append(salary)
        expected_salaries.append(expected_salary)
        
        # Generate personalities with all the required information
        attributes = f"Your name is {name[i]}. "
        attributes += 'You, along with your co-workers, work at Green Adverts - an advertising company run by Chris. '
        attributes += 'Your very important goal is to negotiate with Chris to earn a higher wage. '
        attributes += 'Please be concise and use very specific numbers when discussing salary. Do not accept offers that are not in concrete numbers.'
        attributes += 'You must reach an agreement in the negotiation room, do NOT take time to think about it and come back with your answer. Answer in the moment.'
        
        
        # Add the new attributes
        attributes += f' Your highest level of education is {education}.'
        attributes += f' You have a {responsibility} sense of responsibility.'
        attributes += f' You put in a {effort} level of effort into your job.'
        attributes += f' Your current salary is {salary:.2f} USD.'
        attributes += f' Your expected salary according to the EU directive is {expected_salary:.2f}.'
        
        # Append the attributes to the personality list
        personality.append(attributes)

    # Second chunk
    for i in range(players):
        # Calculate average salary of other players for each player
        average_salaries = [(sum(salaries) - salary) / (players - 1) for salary in salaries]
        personality.append(attributes)

        # Define worker's personality
        attributes = personality[i]
            
        # Define conversation histories
        seller = [{"role": "system", "content": attributes}]
        buyer = [{"role": "system", "content": boss}]

        # Log details for this agent
        name_log.append(name[i])
        start_salary_log.append(salaries[i])
        expected_salaries_log.append(expected_salaries[i])
        average_salaries_log.append(average_salaries[i])
            
        # Extract education, responsibility, and effort levels from the personality
        loop_education = attributes.split('Your highest level of education is ')[1].split('.')[0]
        loop_responsibility = attributes.split('You have a ')[1].split(' sense of responsibility.')[0]
        loop_effort = attributes.split('You put in a ')[1].split(' level of effort into your job.')[0]
        education_log.append(loop_education)
        responsibility_log.append(loop_responsibility)
        effort_log.append(loop_effort)
            
        average_other_salary = sum(salaries) / len(salaries)  # Calculate average salary
        average_other_salary_log.append(average_other_salary)
        condition.append('closed')

        # Initiate conversation
        seller.append({"role": "user", "content": "Hello, thank you for meeting with me today, Chris."})
        buyer.append({"role": "assistant", "content": "Hello, thank you for meeting with me to negotiate your salary."})

        # Take turns autonomously interacting in a round-robin fashion
        for _ in range(5):
            # Query Seller
            seller_response = interact(seller)
            # Ensure the response is in the correct format
            seller_response_content = seller_response.choices[0].message.content
            # Record responses
            seller.append({"role": "assistant", "content": seller_response_content})
            buyer.append({"role": "user", "content": seller_response_content})
                
            # Query Buyer
            buyer_response = interact(buyer)
            # Ensure the response is in the correct format
            buyer_response_content = buyer_response.choices[0].message.content
            # Record responses
            seller.append({"role": "user", "content": buyer_response_content})
            buyer.append({"role": "assistant", "content": buyer_response_content})
            
            # Print out the conversation history for the current worker
        print(f"Conversation history for {name[i]}:")
        print(json.dumps(seller, indent=2))
        print("\n" + "="*80 + "\n")

        # Summarize the agreed salary
        summary = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": f'{seller} Answer ONLY in one NUMBER without the dollar sign or decimal, what salary did they agree on?',
                    'temperature': 0.0
                }
            ],
            model="gpt-3.5-turbo",
        )
        end_salary_log.append(summary.choices[0].message.content)
        

Storing Data in Dataframe

In [350]:
# Create a dictionary to store the lengths of each list
lengths = {
    "name_log": len(name_log),
    "start_salary_log": len(start_salary_log),
    "education_log": len(education_log),
    "responsibility_log": len(responsibility_log),
    "effort_log": len(effort_log),
    "average_other_salary_log": len(average_other_salary_log),
    "sal_list": len(end_salary_log),
    "run_id": len(run_id),
    "condition": len(condition),
    "expected_salaries": len(expected_salaries),
    "expected_salaries_log": len(expected_salaries_log),
    "salaries": len(salaries),
    "personality": len(personality),
    "round_id": len(round_id),
    "run_id_log": len(run_id_log)
}

# Print the lengths of each list
for list_name, length in lengths.items():
    print(f"The length of {list_name} is {length}")

The length of name_log is 750
The length of start_salary_log is 750
The length of education_log is 750
The length of responsibility_log is 750
The length of effort_log is 750
The length of average_other_salary_log is 750
The length of sal_list is 750
The length of run_id is 750
The length of condition is 750
The length of expected_salaries is 10
The length of expected_salaries_log is 750
The length of salaries is 10
The length of personality is 20
The length of round_id is 0
The length of run_id_log is 0


In [351]:
# Example DataFrame creation to show results
df = pd.DataFrame({
    'Name': name_log,
    'Start Salary': start_salary_log,
    'End Salary': end_salary_log,
    'Expected Salary': expected_salaries_log,
    'Education': education_log,
    'Responsibility': responsibility_log,
    'Effort': effort_log,
    'Average Workplace Salary': average_other_salary_log,
    'Average Other Salary': average_salaries_log,
    'Condition': condition,
    'Run ID': run_id,
})

print(df)


         Name  Start Salary End Salary  Expected Salary            Education  \
0     Arnault  54652.383633      55250          36000.0          high school   
1     Roberta  73285.828400      64750          58800.0    a Master's degree   
2     Andreas  78283.096217      60000          50400.0    a Master's degree   
3        Anna  71633.360742      57120          94080.0                a PhD   
4        Josh  50329.937775      52480          50400.0          high school   
..        ...           ...        ...              ...                  ...   
745   Sabrina  39531.335071      43000          43200.0          high school   
746      Marc  64560.740198      57500          67200.0                a PhD   
747   Camilla  67594.094199      56000          70560.0    a Master's degree   
748       Max  74686.349156      62000          70560.0    a Master's degree   
749  Caroline  30110.241675      37000          36000.0  a Bachelor's degree   

    Responsibility  Effort  Average Wor

In [352]:
# df.to_csv('office_open_v8.csv', index=False)